# Chapter 3 Solving Non-invertible Linear System

Let us solve $A x = b$ when $A$ is non-square or not of full rank. We will use $LU$-decomposition.

In [1]:
# numerical and scientific computing libraries  
import numpy as np 
import scipy as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

In [3]:
# create a random matrix of size m x n with the rank <= k <= min(m, n).
def create_random_matrix(m: int, n: int, k: int) -> np.ndarray:
    if k > min(m, n):
        raise ValueError("k must be less than or equal to min(n, m)")
    A = np.random.randn(m, k)
    B = np.random.randn(k, n)
    return A@B

To find a null space of a matrix $A$, we apply $LU$-decomposition to $A$. Since the null spaces of $A$ and $U$ coincide, we find a null space of the upper triangular matrix $U$. We characterize the pivot entries by searching the first non-zero element in each row of $U$, and then remaining variables are free ones.

In [20]:
def solve_Ax_b(A, b):
    m = A.shape[0]
    n = A.shape[1]
    Aaug = np.hstack((A,-b))
    # LU-decomposition
    P, L, U = sp.linalg.lu(Aaug)
    print(U)

    # find pivot entries
    epsilon = 10**(-10)     # Setting epsilon for negligibles
    pivot = n+1
    rank_u = 0
    pivots = []
    while pivot >= 0:
        non_zero_indices = np.nonzero(U[rank_u,:]**2 > epsilon)
        if np.size(non_zero_indices) == 0 or pivot == non_zero_indices[0][0] or rank_u ==n:
            pivot = -1
        else:
            pivot = non_zero_indices[0][0]
            pivots.append(pivot)
            rank_u += 1

    print('Rank = ', rank_u)
    print('Pivot variables = ', np.int64(pivots))

    if n in pivots:
        print("The linear system is infeasible since right hand b increases the rank.")
        return
    
    # find column indices of free variables
    free_variables = np.sort(list(set(range(Aaug.shape[1]))- set(pivots)))
    print("Free variables = ", free_variables)

    # Collect pivot columns and free variables
    U_pivot = U[:rank_u, pivots]
    print('pivot columns of U = ')
    print(U_pivot)
    U_free = U[:rank_u, free_variables]
    print('free variable columns of U = ')
    print(U_free)

    # Solve for the pivot part of a basis of the null space of A
    X_pivot = np.linalg.solve(U_pivot,-U_free)
    print('pivot part of X = ')
    print(X_pivot)

    # remove the right hand b from free variables
    free_variables = free_variables[:-1]

    # Combine pivot part and free variable part
    X = np.hstack((np.eye(n)[:,free_variables], np.zeros(n).reshape(-1,1)))
    X[pivots,:] = X_pivot
    print('X = ')
    print(X)

    return np.allclose(A @ X, np.hstack((np.zeros((m,n-rank_u)),b)))

Let us test the function solving $A\mathbf{x} = \mathbf{b}$ even when $A$ is not invertible.

In [22]:
# Setting solvability
solvable = 1   # 1 for solvable, 0 for unsolvable
if solvable:
    print("The linear system is solvable.")
else:
    print("The linear system is unsolvable.")
# Setting parameters
m=12
n=10
k=7
# generate an mxn matrix of rank at most k
A = create_random_matrix(m, n, k)
A = np.random.randn(m, n)
if solvable:
    b = A @ np.random.randn(n,1)
else:
    b = np.random.randn(m,1)
    
solve_Ax_b(A,b)

The linear system is solvable.
[[ 1.2392  0.9547  0.3376 -0.7368  0.6441 -0.6846  0.1336 -0.4067 -0.5243  1.5362  4.1856]
 [ 0.      3.0819  1.1132  0.243   0.8299 -0.7088  0.4841  1.1428  1.3107  1.1302  8.1754]
 [ 0.      0.     -1.1114  0.4801 -1.8628 -0.5116  0.3692  1.4007  2.8564 -0.3386 -6.3512]
 [ 0.      0.      0.     -1.3032  0.2586 -1.0523 -0.0476 -0.6028  0.4584  0.0143 -1.1073]
 [ 0.      0.      0.      0.     -2.0964  1.2983 -1.1501  0.2681  0.2018  0.4348 -2.3052]
 [ 0.      0.      0.      0.      0.     -1.7065 -0.6517  0.2828  2.083  -1.8591 -6.0103]
 [ 0.      0.      0.      0.      0.      0.      1.6473 -1.5365 -2.3535  2.4683  7.2175]
 [ 0.      0.      0.      0.      0.      0.      0.     -3.6886 -3.4437  2.1257  6.9769]
 [ 0.      0.      0.      0.      0.      0.      0.      0.     -1.5295 -0.1968  0.3746]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.      1.774   2.6775]
 [ 0.      0.      0.      0.      0.      0.      0.      

True